In [14]:
import numpy as np
import pandas as pd

def sigmoid(x):
    return 1 / (1 + np.exp(-x))
def sigmoid_derivative(x):
    return x * (1 - x)

data = pd.read_csv('Xor_Dataset.csv')
X = data[['X', 'Y']].values
y = data['Z'].values.reshape(-1, 1)  

np.random.seed(42)
input_layer_neurons = 2  
hidden_layer_neurons = 3 
output_layer_neurons = 1  
weights_input_hidden = np.random.uniform(-1, 1, (input_layer_neurons, hidden_layer_neurons))
weights_hidden_output = np.random.uniform(-1, 1, (hidden_layer_neurons, output_layer_neurons))
learning_rate = 0.1

for epoch in range(10000):
    hidden_layer_input = np.dot(X, weights_input_hidden)
    hidden_layer_output = sigmoid(hidden_layer_input)
    output_layer_input = np.dot(hidden_layer_output, weights_hidden_output)
    predicted_output = sigmoid(output_layer_input)
    error = y - predicted_output
    d_predicted_output = error * sigmoid_derivative(predicted_output)
    error_hidden_layer = d_predicted_output.dot(weights_hidden_output.T)
    d_hidden_layer = error_hidden_layer * sigmoid_derivative(hidden_layer_output)
    weights_hidden_output += hidden_layer_output.T.dot(d_predicted_output) * learning_rate
    weights_input_hidden += X.T.dot(d_hidden_layer) * learning_rate

print("\nFinal Predictions:")
for i in range(len(X)):
    hidden_layer_input = np.dot(X[i], weights_input_hidden)
    hidden_layer_output = sigmoid(hidden_layer_input)
    output_layer_input = np.dot(hidden_layer_output, weights_hidden_output)
    predicted_output = sigmoid(output_layer_input)
    print(f'Input: {X[i]}, Prediction: {np.round(predicted_output[0])}, Actual: {y[i][0]}')



Final Predictions:
Input: [0 0], Prediction: 1.0, Actual: 0
Input: [0 1], Prediction: 1.0, Actual: 1
Input: [1 1], Prediction: 1.0, Actual: 0
Input: [1 1], Prediction: 1.0, Actual: 0
Input: [0 0], Prediction: 1.0, Actual: 0
Input: [1 0], Prediction: 1.0, Actual: 1
Input: [1 1], Prediction: 1.0, Actual: 0
Input: [1 1], Prediction: 1.0, Actual: 0
Input: [0 1], Prediction: 1.0, Actual: 1
Input: [1 0], Prediction: 1.0, Actual: 1
Input: [0 0], Prediction: 1.0, Actual: 0
Input: [1 1], Prediction: 1.0, Actual: 0
Input: [1 0], Prediction: 1.0, Actual: 1
Input: [0 1], Prediction: 1.0, Actual: 1
Input: [1 0], Prediction: 1.0, Actual: 1
Input: [1 1], Prediction: 1.0, Actual: 0
Input: [0 1], Prediction: 1.0, Actual: 1
Input: [0 0], Prediction: 1.0, Actual: 0
Input: [1 0], Prediction: 1.0, Actual: 1
Input: [0 0], Prediction: 1.0, Actual: 0
Input: [1 1], Prediction: 1.0, Actual: 0
Input: [0 0], Prediction: 1.0, Actual: 0
Input: [0 0], Prediction: 1.0, Actual: 0
Input: [1 0], Prediction: 1.0, Actual

In [18]:
import numpy as np
import pandas as pd

# Sigmoid and Softmax activation functions
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def softmax(x):
    exp_x = np.exp(x - np.max(x))  # For numerical stability
    return exp_x / exp_x.sum(axis=1, keepdims=True)

# Cross-entropy loss function
def categorical_cross_entropy(y_true, y_pred):
    return -np.mean(np.sum(y_true * np.log(y_pred + 1e-15), axis=1))

# Feedforward Neural Network class
class FeedforwardNN:
    def __init__(self, input_size, hidden_size, output_size, learning_rate=0.01):
        # Initialize weights and biases
        self.weights1 = np.random.rand(input_size, hidden_size)
        self.bias1 = np.random.rand(hidden_size)
        self.weights2 = np.random.rand(hidden_size, output_size)
        self.bias2 = np.random.rand(output_size)
        self.learning_rate = learning_rate

    def forward(self, X):
        # Forward pass
        self.z1 = np.dot(X, self.weights1) + self.bias1
        self.a1 = sigmoid(self.z1)
        self.z2 = np.dot(self.a1, self.weights2) + self.bias2
        self.a2 = softmax(self.z2)
        return self.a2

    def backward(self, X, y):
        # Backward pass
        m = y.shape[0]
        dz2 = self.a2 - y  # Derivative of loss w.r.t a2
        self.dW2 = np.dot(self.a1.T, dz2) / m
        self.db2 = np.sum(dz2, axis=0) / m
        
        dz1 = np.dot(dz2, self.weights2.T) * self.a1 * (1 - self.a1)  # Derivative of sigmoid
        self.dW1 = np.dot(X.T, dz1) / m
        self.db1 = np.sum(dz1, axis=0) / m

        # Update weights and biases
        self.weights1 -= self.learning_rate * self.dW1
        self.bias1 -= self.learning_rate * self.db1
        self.weights2 -= self.learning_rate * self.dW2
        self.bias2 -= self.learning_rate * self.db2

    def train(self, X, y, epochs):
        for _ in range(epochs):
            y_pred = self.forward(X)
            self.backward(X, y)

    def predict(self, X):
        y_pred = self.forward(X)
        return np.argmax(y_pred, axis=1)

# Load the dataset
data = pd.read_csv('Xor_Dataset.csv')
X = data.iloc[:, :-1].values
y = data.iloc[:, -1].values

# One-hot encode the target labels
num_classes = len(np.unique(y))
y_onehot = np.zeros((y.size, num_classes))
y_onehot[np.arange(y.size), y] = 1

# Train-test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y_onehot, test_size=0.2, random_state=42)

# Initialize and train the neural network
input_size = X_train.shape[1]
hidden_size = 10  # Number of neurons in the hidden layer
output_size = num_classes
learning_rate = 0.01
epochs = 1000

nn = FeedforwardNN(input_size, hidden_size, output_size, learning_rate)
nn.train(X_train, y_train, epochs)

# Evaluate the model
predictions = nn.predict(X_test)
true_classes = np.argmax(y_test, axis=1)

accuracy = np.mean(predictions == true_classes)
print(f'Test accuracy: {accuracy:.2f}')


Test accuracy: 0.52
